<a href="https://colab.research.google.com/github/bingxiaochen/ST-554-Project1/blob/main/Task1/ST_554_P1_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Bing Chen

Purpose: This programs is written for task 1 of project 1.

Date: 2-10-2026

# Read in data

To use the air quality data set available at the UCI machine learning repository, we want to first install the `ucimlrepo` library.

In [74]:
!pip install ucimlrepo

After installing it, import it in to use it. And, go ahead to import all other modules we might use.

In [75]:
import ucimlrepo as uci
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats


Now, we are ready to read in the air quality data from the library.

In [76]:
air_quality = uci.fetch_ucirepo(id=360)
air_quality.data.features

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


Now, let's do some data cleaning.

Let's check the type of each column.



In [77]:
air_quality.data.features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   float64
 3   PT08.S1(CO)    9357 non-null   int64  
 4   NMHC(GT)       9357 non-null   int64  
 5   C6H6(GT)       9357 non-null   float64
 6   PT08.S2(NMHC)  9357 non-null   int64  
 7   NOx(GT)        9357 non-null   int64  
 8   PT08.S3(NOx)   9357 non-null   int64  
 9   NO2(GT)        9357 non-null   int64  
 10  PT08.S4(NO2)   9357 non-null   int64  
 11  PT08.S5(O3)    9357 non-null   int64  
 12  T              9357 non-null   float64
 13  RH             9357 non-null   float64
 14  AH             9357 non-null   float64
dtypes: float64(5), int64(8), object(2)
memory usage: 1.1+ MB


In [78]:
air_quality.data.features.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,9.778305,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,43.203623,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,10.900000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,17.200000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,24.100000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000


Now, we want to remove any observations where the `C6H6(GT)` or `PT08.S1(CO)` are -200 as these represent missing values. We will save the result as a new dataset. We can compare the shape of these datasets to see how many observations were removed.

In [79]:
air_clean = air_quality.data.features.loc[
    (air_quality.data.features["C6H6(GT)"] != -200) &
    (air_quality.data.features["PT08.S1(CO)"] != -200)
]

print(air_quality.data.features.shape)
print(air_clean.shape)


(9357, 15)
(8991, 15)


In [80]:
air_clean.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000,8991.000000
mean,-34.897053,1099.833166,-158.702591,10.083105,939.153376,163.745078,835.493605,56.666555,1456.264598,1022.906128,18.317829,49.234201,1.025530
std,78.202259,217.080037,140.680200,7.449820,266.831429,252.090703,256.817320,126.910900,346.206794,398.484288,8.832116,17.316892,0.403813
min,-200.000000,647.000000,-200.000000,0.100000,383.000000,-200.000000,322.000000,-200.000000,551.000000,221.000000,-1.900000,9.200000,0.184700
25%,0.600000,937.000000,-200.000000,4.400000,734.500000,49.000000,658.000000,52.000000,1227.000000,731.500000,11.800000,35.800000,0.736800
50%,1.500000,1063.000000,-200.000000,8.200000,909.000000,140.000000,806.000000,96.000000,1463.000000,963.000000,17.800000,49.600000,0.995400
75%,2.600000,1231.000000,-200.000000,14.000000,1116.000000,280.000000,969.500000,132.000000,1674.000000,1273.500000,24.400000,62.500000,1.313700
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,333.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000


Looks like 366 observations are removed.

Now, we are ready to make predictions of C6H6(GT).

# Prediction of C6H6(GT)

We want to be able to predict a value of `C6H6(GT)` for a new sample. We'll quantify the quality of the prediction through a loss function.


So, first, let's understand what a loss function is.

Suppose there is some prediction (call it $c$). The squared error loss for a data point (call it $y_1$) and $c$ is $L(y_1, c) = (y_1 − c)^2$

For a given set of data $(y_1,..., y_n)$, we could consider the mean squared error:

Root Mean Square Error = $RMSE(c) = \sqrt{\frac{1}{n}\sum^n_{i=1}(y_1 − c)^2}$


As a prediction, we now want to choose the value of $c$ that minimizes this equation.



We'll use numerical methods to find the optimal value:

1. Grid search
2. Gradient descent

We'll implement these two algorithms for two equations representing $c$:

- we don’t consider any data other than  𝑦 , and c is just a constant that minimizes the function.
- we consider a linear equation (SLR model) of one other numeric variable.


## Grid Search

Grid search method follows the steps:  

1. Create a grid of values for $c$.
2. Find the RMSE for each value of $c$.
3. Determine which value of $c$ gives the optimal (smallest) RMSE.
4. And finally, report that as the prediction!


Now let's implement it together.



### Just y

We don't consider any data other than the $y$, then $c$ will just be some constant that minimises the function. To use a reasonable grid for $c$, let's consider using the first and third quartiles for the `C6H6(GT)`, since $c$ should live where most of the data are.


In [81]:
q1 = air_clean["C6H6(GT)"].quantile(0.25)
q3 = air_clean["C6H6(GT)"].quantile(0.75)

q1, q3

(np.float64(4.4), np.float64(14.0))

Now, create a grid of $c$ values using the quartiles. Let's create 1000 evenly spaced candidate values for $c$.

In [82]:
c_grid = np.linspace(q1, q3, 1000)

Next step is to find the RMSE for each value of c. Let's create a function to do that.

In [83]:
def rmse(y, c):
    """
    Find the RMSE for a given y and c.
    """
    y = np.asarray(y)
    return np.sqrt(np.mean([(yi - c) ** 2 for yi in y]))

Let's test out the function before moving to the next step. Pass the $c$ grid we created into the function.

In [84]:
rmse_values = [rmse(air_clean["C6H6(GT)"], c) for c in c_grid]
rmse_values

[np.float64(9.369702603734977),
 np.float64(9.363877101921979),
 np.float64(9.358057841590487),
 np.float64(9.35224483439144),
 np.float64(9.346438091992221),
 np.float64(9.340637626076646),
 np.float64(9.334843448344916),
 np.float64(9.329055570513606),
 np.float64(9.32327400431563),
 np.float64(9.31749876150021),
 np.float64(9.311729853832851),
 np.float64(9.305967293095314),
 np.float64(9.300211091085576),
 np.float64(9.29446125961781),
 np.float64(9.28871781052235),
 np.float64(9.28298075564566),
 np.float64(9.2772501068503),
 np.float64(9.271525876014898),
 np.float64(9.265808075034117),
 np.float64(9.26009671581862),
 np.float64(9.254391810295033),
 np.float64(9.248693370405926),
 np.float64(9.243001408109757),
 np.float64(9.237315935380852),
 np.float64(9.231636964209374),
 np.float64(9.225964506601269),
 np.float64(9.220298574578244),
 np.float64(9.21463918017773),
 np.float64(9.20898633545284),
 np.float64(9.203340052472333),
 np.float64(9.197700343320575),
 np.float64(9.19206

Good, looks like it's working correctly. Now, use a for loop to find which value of $c$ gives the smallest RMSE. We initialize with the first RMSE and its $c$, loop through the rest, every time we find a smaller RMSE, we update both values. Note that `rmse[i] corresponds to c_grid[i]`.

In [85]:
smallest_rmse = rmse_values[0]
best_c = c_grid[0]

for i in range(1, len(rmse_values)):
    if rmse_values[i] < smallest_rmse:
        smallest_rmse = rmse_values[i]
        best_c = c_grid[i]

smallest_rmse, best_c

(np.float64(7.449406376152314), np.float64(10.07927927927928))

There! $c = 10.07927927927928$ is our prediction!

Now, let's wrap all of that in a function.

In [86]:
def grid_search(y, c_grid):

    """
    Find the the optimal constant prediction for your response variable.

    Parameters
    ----------
    y : array-like
        The response variable.
    c_grid : array-like
        The grid of c values.
    """

    rmse_values = [rmse(y, c) for c in c_grid]

    smallest_rmse = rmse_values[0]
    best_c = c_grid[0]

    for i in range(1, len(rmse_values)):
        if rmse_values[i] < smallest_rmse:
            smallest_rmse = rmse_values[i]
            best_c = c_grid[i]

    return best_c, print(str(best_c) + " is your optimal constant prediction!")



Test our function with `C6H6(GT)` and `PT08.S1(CO)` variable.  

In [87]:
grid_search(air_clean["C6H6(GT)"], c_grid)

10.07927927927928 is your optimal constant prediction!


(np.float64(10.07927927927928), None)

The calculus based answer for this should comes out to be the sample mean. Let's check!

In [88]:
np.mean(air_clean["C6H6(GT)"])

np.float64(10.083105327549772)

That's very close!

Update our $c$ grid with values between Q1 and Q3 of `PT08.S1(CO)` variable in order to make a prediction on it.

In [89]:
c_grid_pt08 = np.linspace(air_clean["PT08.S1(CO)"].quantile(0.25),
                          air_clean["PT08.S1(CO)"].quantile(0.75), 1000)

In [90]:
grid_search(air_clean["PT08.S1(CO)"], c_grid_pt08)

1099.7447447447448 is your optimal constant prediction!


(np.float64(1099.7447447447448), None)

In [91]:
np.mean(air_clean["PT08.S1(CO)"])

np.float64(1099.8331664998332)

Great! Now let's implement grid search method using y and another numeric variable x.

### y and x
We'll use grid search to find the optimal
pair of values for $b_0$ (intercept) and $b_1$ (slope) using `PT08.S1(CO)` as our $x$ variable and `C6H6(GT)` as our $y$ variable. For each observation ($i$), our prediction is given by $c_i = b_0 + b_1x_i$. So, note that our grid now has two-dimensions.

This time, consider a default grid of $b_0$ and $b_1$ values as follows:
- Use $b_0$ values from -25 to -15 with increments of 0.1.
- Use $b_1$ values from -5 to 5 with increments of 0.01


Let's create a function that takes in an x, y column, start, stop, and step for slope and intercept with default above and outputs the optimal values (optimal $b_0$ and $b_1$ combination).

The RMSE formula is a bit different this time since $c_i = b_0 + b_1x_i$. So, the updated RMSE formula is: $RMSE(c) = \sqrt{\frac{1}{n}\sum^n_{i=1}(y_i − (b_0 + b_1x_i)^2}$


Then we will use these values to predict a new `C6H6(GT)` for a `PT08.S1(CO)` of 946, 1075, and 1246.

In [92]:
def grid_search_SLR(y,
                    x,
                    start_b0 = -25,
                    start_b1 = -5,
                    step_size_b0=0.1,
                    step_size_b1=0.01,
                    end_b0 = -15,
                    end_b1 = 5):

    """
    Find the the optimal intercept and slope for your linear equation of
    the response and explanatory variable.

    Parameters
    ----------
    y : array-like
        The response variable.
    x : array-like
        The explanatory variable.
    start_b0 : float
        Starting value for intercept.
    start_b1 : float
        Starting value for slope.
    step_size_b0 : float
        Step size for intercept.
    step_size_b1 : float
        Step size for slope.
    end_b0 : float
        Ending value for intercept.
    end_b1 : float
        Ending value for slope.
    """


    y = np.asarray(y)
    x = np.asarray(x)

    b0_grid = np.arange(start_b0, end_b0, step_size_b0)
    b1_grid = np.arange(start_b1, end_b1, step_size_b1)

    rmse= [
    np.sqrt(np.mean((y - (b0 + b1 * x)) ** 2))
    for b0 in b0_grid
    for b1 in b1_grid
    ]

    # create coefficient pairs
    params = [(b0, b1) for b0 in b0_grid for b1 in b1_grid]

    # initialize the smallest rmse, best coefficients
    smallest_rmse = rmse[0]
    best_b0, best_b1 = params[0]

    # find best parameters
    for i in range(1, len(rmse)):
        if rmse[i] < smallest_rmse:
            smallest_rmse = rmse[i]
            best_b0, best_b1 = params[i]

    print(f"(b0, b1) = ({best_b0}, {best_b1}) is your optimal linear prediction!")

    return best_b0, best_b1, smallest_rmse



Now, let's test it!


In [93]:
grid_search_SLR(air_clean["C6H6(GT)"], air_clean["PT08.S1(CO)"])

(b0, b1) = (-22.89999999999997, 0.02999999999989278) is your optimal linear prediction!


(np.float64(-22.89999999999997),
 np.float64(0.02999999999989278),
 np.float64(3.4861424255520586))

Let's check our result with simple linear regression estimates from `SciPy.stats`.

In [94]:
x = air_clean["PT08.S1(CO)"].values
y = air_clean["C6H6(GT)"].values
results = stats.linregress(x, y)

results.intercept, results.slope

(np.float64(-23.2752218997247), np.float64(0.03033035213280189))

They are in close agreeement! Let's use the coefficient estimates to predict a new `C6H6(GT)` for a `PT08.S1(CO)` of 946, 1075, and 1246. Recall $c_i = b_0 + b_1x_i$.

In [95]:
# x = 946
-22.89999999999997+ 0.02999999999989278*946

5.479999999898599

In [96]:
# x = 1075
-22.89999999999997+ 0.02999999999989278*1075

9.349999999884766

In [97]:
# x = 1246
-22.89999999999997+ 0.02999999999989278*1246

14.479999999866429

## Gradient Descent

The basic idea of gradient descent is to starts with an initial guess and then repeatedly adjusts that guess by moving in the negative direction of the function's slope. The size of each adjustment is controlled by a step size, and the process continues until the changes become very small or a maximum number of steps is reached. In this case, we use gradient descent to find the value of $c$ that minimizes the RMSE.

### Just y

First, consider the case where we only use $y$ and no $x$. We can reuse the `rmse` function we creaeted earlier. Then, we need a function for the difference quotient (numerical derivative).

In [98]:
def diff_quotient(y, c, delta=0.001):
    return (rmse(y, c + delta) - rmse(y, c)) / delta

We want to use a while loop (we stop when movement < tolerance, or when `max_iters` is reached).


In [99]:
def gradient_descent_constant(y, start_c, step_size=0.01, num_tol=0.0001, delta=0.001, max_iters=10000):
    """
    Gradient descent to minimize RMSE and return the final c.

    Parameters
    ----------
    y : array-like
        Response variable.
    start_c : float
        Initial guess.
    step_size : float
        Step size.
    num_tol : float
        Numeric tolerance.
    delta : float
        Small change for difference quotient derivative approximation.
    max_iters : int
         safety that stops the loop after the maximum number of iterations is reached.
    """

    y = np.asarray(y)

    cur_c = float(start_c)
    iters = 0

    while iters < max_iters:
        new_c = cur_c - diff_quotient(y, cur_c, delta=delta)  * step_size

        if abs(new_c - cur_c) < num_tol:
            cur_c = new_c
            break

        cur_c = new_c
        iters += 1

    return cur_c

Let's try out the function. I chose delta of 0.001, step size 0.01, and tolerance 0.0001.

In [100]:
gradient_descent_constant(air_clean["C6H6(GT)"], start_c=0, step_size=0.01, num_tol=0.0001, max_iters=5000)


np.float64(10.008241234506805)

From calculus, we know this comes out to be the sample mean, $\bar{y}$, let's check!

In [101]:
air_clean["C6H6(GT)"].mean()

np.float64(10.083105327549772)

It's very close! The minor difference may be due to our tolerance level. If we set it even smaller, our result will get closer and closer to the true value.


Now let's run it using the `PT08.S1(CO)` variable as the response and 1100 as our starting point. For this time, I chose delta of 0.001, step size 0.1, and tolerance 0.00001.

In [102]:
ybar = air_clean["PT08.S1(CO)"].mean()
pred = gradient_descent_constant(air_clean["PT08.S1(CO)"], start_c=1100, step_size=0.1, num_tol=0.00001)

ybar, pred

(np.float64(1099.8331664998332), np.float64(1099.8543562566488))

Looks good! That's close!

Now let's implement the algorithm using y and another numeric variable x.

### y and x

We'll find the optimal value of $b_0$ and $b_1$ using `PT08.S1(CO)` as our explanatory variable and `C6H6(GT)` as our response variable. We follow similar steps as the above algorithm. First, we will derive a RMSE function.

In [103]:
def rmse_slr(y, x, b0, b1):
    return np.sqrt(np.mean((y - (b0 + b1 * x)) ** 2))

Now, since we're optimazing over 2 variables, $b_0$ and $b_1$, we need a difference of quotient function for each variable.

In [104]:
# Difference quotient for b0
def diff_quotient_b0(y, x, b0, b1, delta0=0.005):
    return (rmse_slr(y, x, b0 + delta0, b1) -
            rmse_slr(y, x, b0, b1)) / delta0


# Difference quotient for b1
def diff_quotient_b1(y, x, b0, b1, delta1=0.005):
    return (rmse_slr(y, x, b0, b1 + delta1) -
            rmse_slr(y, x, b0, b1)) / delta1

Now we have everything we need to write our gradient descent function.

In [105]:
def gradient_descent_slr(y, x,
                         start_b0,
                         start_b1,
                         step_size_b0=0.01,
                         step_size_b1=0.001,
                         tol=0.0001,
                         delta_b0 = 0.001,
                         delta_b1 = 0.001,
                         max_iters=100000):

    """
    Gradient descent to minimize RMSE and return the final slope and intercept.

    Parameters
    ----------
    y : array-like
        Response variable.
    x : array-like
        Explanatory variable.
    start_b0 : float
        Starting value for intercept.
    start_b1 : float
        Starting value for slope.
    step_size_b0 : float
        Step size for intercept.
    step_size_b1 : float
        Step size for slope.
    tol : float
        Numeric tolerance.
    delta_b0 : float
        Small change for difference quotient derivative approximation for intercept.
    delta_b1 : float
        Small change for difference quotient derivative approximation for slope.
    max_iters : int
         safety that stops the loop after the maximum number of iterations is reached.
    """

    y = np.asarray(y)
    x = np.asarray(x)

    cur_b0 = start_b0
    cur_b1 = start_b1

    iters = 0

    while iters < max_iters:

        slope_b0 = diff_quotient_b0(y, x, cur_b0, cur_b1, delta_b0)
        slope_b1 = diff_quotient_b1(y, x, cur_b0, cur_b1, delta_b1)


        new_b0 = cur_b0 - slope_b0 * step_size_b0
        new_b1 = cur_b1 - slope_b1 * step_size_b1

        movement = np.linalg.norm(np.array([new_b0, new_b1]) - np.array([cur_b0, cur_b1]))

        if movement < tol:
            cur_b0, cur_b1 = new_b0, new_b1
            break

        cur_b0, cur_b1 = new_b0, new_b1
        iters += 1

    return cur_b0, cur_b1

Now, let's test the function with initial starting values of -20 for the intercept and 0 for the slope. I chose a step size of 0.5 for the intercept, 0.000005 for the slope, a delta of 0.005 for intercept and 0.000005 for slope, and a tolerance of 0.00001.


In [106]:
b0_est, b1_est = gradient_descent_slr(
    y = air_clean["C6H6(GT)"],
    x = air_clean["PT08.S1(CO)"],
    start_b0=-20,
    start_b1=0,
    step_size_b0=0.5,
    step_size_b1=0.000005,
    tol =  0.00001,
    delta_b0=0.005,
    delta_b1=0.000005,
    max_iters=100000
)

b0_est, b1_est

(np.float64(-23.266574391895755), np.float64(0.030320279515369645))

Recall from previous, the true intercept and slope is -23.2752218997247, 0.03033035213280189 respectively. Our algorithm took us very close!

Lastly, let's use our coefficient estimates to predict a new C6H6(GT) for a PT08.S1(CO) of 946, 1075, and 1246.


In [107]:
# x = 946
b0_est+ b1_est*946

np.float64(5.416410029643927)

In [108]:
# x = 1075
b0_est+ b1_est*1075

np.float64(9.32772608712661)

In [109]:
# x = 1246
b0_est+ b1_est*1246

np.float64(14.51249388425482)